In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriatic_Arthritis/GSE57383'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of CD14+ cells from RA, PsA and PsO patients with Infliximab treatment"
!Series_summary	"objection: The immune inflammatory disorders rheumatoid arthritis (RA), psoriatic arthritis (PsA) and psoriasis (Ps) share common pathologic features and show responsiveness to anti-tumor necrosis factor (TNF) agents yet they are phenotypically distinct. The aim of this study was to examine if anti-TNF therapy is associated with divergent gene expression profiles in circulating cells and target tissues of patients with these diseases"
!Series_summary	"Method: Peripheral blood CD14+ and CD14- cells were isolated from 9 RA, 12 PsA and 10 Ps patients before and after infliximab (IFX) treatment"
!Series_overall_design	"Between April 2007 and June 2009, 31 patients with active RA, PsA and Ps who were naïve to anti-TNF agents, were recruited from the Faculty Rheumatology Clinics at the University of Rochester Medical Center after informed, written co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the series title and summary, it appears to be gene expression data

# Determine data availability for 'Psoriatic_Arthritis', 'age', and 'gender'
trait_row = 6  # 'disease: Health Control', 'disease: Psoriasis', 'disease: Psoriatic Arthritis', 'disease: Rheumatoid Arthritis'
age_row = 2  # 'age: 40', 'age: 54', 'age: 36', etc.
gender_row = 1  # 'Sex: F', 'Sex: M'

# Define conversion functions for each variable

def convert_trait(value):
    trait_map = {'Psoriatic Arthritis': 1, 'Health Control': 0, 'Psoriasis': None, 'Rheumatoid Arthritis': None}
    v = value.split(': ')[1]
    return trait_map.get(v, None)

def convert_age(value):
    try:
        v = value.split(': ')[1]
        return float(v)
    except ValueError:
        return None

def convert_gender(value):
    gender_map = {'F': 0, 'M': 1}
    v = value.split(': ')[1]
    return gender_map.get(v, None)

# Save cohort information
save_cohort_info('GSE57383', './preprocessed/Psoriatic_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriatic_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriatic_Arthritis/trait_data/GSE57383.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1381524': [0.0, 40.0, 0.0], 'GSM1381525': [0.0, 40.0, 0.0], 'GSM1381526': [0.0, 54.0, 1.0], 'GSM1381527': [0.0, 36.0, 0.0], 'GSM1381528': [0.0, 23.0, 0.0], 'GSM1381529': [0.0, 42.0, 0.0], 'GSM1381530': [0.0, 24.0, 0.0], 'GSM1381531': [0.0, 23.0, 1.0], 'GSM1381532': [0.0, 62.0, 1.0], 'GSM1381533': [0.0, 46.0, 1.0], 'GSM1381534': [0.0, 56.0, 0.0], 'GSM1381535': [0.0, 32.0, 0.0], 'GSM1381536': [0.0, 47.0, 0.0], 'GSM1381537': [0.0, 60.0, 0.0], 'GSM1381538': [0.0, 44.0, 0.0], 'GSM1381539': [0.0, 46.0, 0.0], 'GSM1381540': [0.0, 36.0, 0.0], 'GSM1381541': [0.0, 64.0, 1.0], 'GSM1381542': [0.0, 23.0, 1.0], 'GSM1381543': [nan, 19.0, 0.0], 'GSM1381544': [nan, 24.0, 0.0], 'GSM1381545': [nan, 46.0, 1.0], 'GSM1381546': [nan, 61.0, 1.0], 'GSM1381547': [nan, 57.0, 1.0], 'GSM1381548': [nan, 57.0, 1.0], 'GSM1381549': [nan, 35.0, 0.0], 'GSM1381550': [nan, 53.0, 1.0], 'GSM1381551': [nan, 59.0, 0.0], 'GSM1381552': [nan, 53.0, 1.0], 'GSM1381553': [nan, 44.0, 0.0], 'GSM1381554': [nan, 59.0, 0.0], 'GSM138

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Based on the outputs of STEP3 and STEP5, determine identifier_key and gene_symbol_key
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Psoriatic_Arthritis/gene_data/GSE57383.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Psoriatic_Arthritis')

# 4. Save the cohort information.
save_cohort_info('GSE57383', './preprocessed/Psoriatic_Arthritis/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Psoriatic_Arthritis/GSE57383.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Psoriatic_Arthritis', the least common label is '0.0' with 19 occurrences. This represents 34.55% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.0
  50% (Median): 46.0
  75%: 60.0
Min: 23.0
Max: 70.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 43.64% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

